In [134]:
import pandas as pd
import numpy as np
import re

__데이터로드__

In [135]:
path='D:/세미 1차 텍스트 마이닝/formats/'
bt = pd.read_csv(path+'Biden_twitter.csv')
p2b = pd.read_csv(path+'ppl2Biden_twitter.csv')
tt = pd.read_csv(path+'Trump_twitter.csv')
p2t = pd.read_csv(path+'ppl2Trump_twitter.csv')

__중복제거__

In [136]:
to_Biden = p2b.drop_duplicates()
from_Biden = bt.drop_duplicates()

to_Trump = p2t.drop_duplicates()
from_Trump = tt.drop_duplicates()

__정상 트윗 여부 칼럼 추가 (트위터 법 위반 어쩌구 텍스트가 있는 행들은 '·' 이게 없음)__

In [137]:
def valid_tweet(row):
    text_list = row.values[0].split('\n')  
    try:
        dot_position = text_list[1:4].index('·') # 요고
        return True
    except ValueError:
        return False

In [138]:
from_Biden = pd.concat([from_Biden,pd.Series(from_Biden.apply(valid_tweet,axis=1),name='valid')],axis=1)
to_Biden = pd.concat([to_Biden,pd.Series(to_Biden.apply(valid_tweet,axis=1),name='valid')],axis=1)

from_Trump = pd.concat([from_Trump,pd.Series(from_Trump.apply(valid_tweet,axis=1),name='valid')],axis=1)
to_Trump = pd.concat([to_Trump,pd.Series(to_Trump.apply(valid_tweet,axis=1),name='valid')],axis=1)

__각 데이터 별 비정상 데이터 수__

In [139]:
(from_Biden.valid==False).sum() , (to_Biden.valid==False).sum()

(0, 7)

In [140]:
(from_Trump.valid==False).sum() , (to_Trump.valid==False).sum() 

(4, 7)

__정상 데이터만 남기기__

In [141]:
from_Biden = from_Biden[from_Biden['valid']==True]
to_Biden = to_Biden[to_Biden['valid']==True]

from_Trump = from_Trump[from_Trump['valid']==True]
to_Trump = to_Trump[to_Trump['valid']==True]

__텍스트 파싱하기 (date , text) 만 관심 사항__

In [142]:
def parse_text_crop(text):
    text_list = text.split('\n') 

    dot_position = text_list[1:4].index('·')  

    date = text_list[dot_position + 2] 

    
    if " 님에게 보내는 답글" in text_list:
        idx = text_list.index(" 님에게 보내는 답글")+1
    else:
        idx = dot_position+2
    
    text=''
    if idx<len(text_list):
        
        for i in range(idx,len(text_list)):
            if '@' in text_list[i]:
                continue
            if '월' in text_list[i]:
                continue
            text+= text_list[i]

    return pd.Series([date, text])

In [143]:
to_Biden_parsed = pd.DataFrame()
to_Biden_parsed[['date', 'text']] = to_Biden['all_text'].apply(parse_text_crop)


from_Biden_parsed = pd.DataFrame()
from_Biden_parsed[['date', 'text']] = from_Biden['all_text'].apply(parse_text_crop)


to_Trump_parsed = pd.DataFrame()
to_Trump_parsed[['date', 'text']] = to_Trump['all_text'].apply(parse_text_crop)


from_Trump_parsed = pd.DataFrame()
from_Trump_parsed[['date', 'text']] = from_Trump['all_text'].apply(parse_text_crop)

__혹시나 결측 있나__

In [144]:
to_Biden_parsed.isnull().sum() , from_Biden_parsed.isnull().sum()

(date    0
 text    0
 dtype: int64,
 date    0
 text    0
 dtype: int64)

In [145]:
to_Trump_parsed.isnull().sum() , from_Trump_parsed.isnull().sum()

(date    0
 text    0
 dtype: int64,
 date    0
 text    0
 dtype: int64)

__date 칼럼을 우선 date 객체로 만들고 , datetime64[ns]로 다시 바꿔서 dt 접근 가능하게 만들어서 9월 이전 데이터 짜르기__  
(간혹 3,7월 데이터가 몇개 섞여있어서 뺌)

In [146]:
from datetime import date

def as_datetime(element):
    dates=re.findall('\d+',element)
    if len(dates)>2:
        dates = dates[1:]
    ret_val = date(2020,int(dates[0]),int(dates[1]))

    return ret_val

In [147]:
to_Biden_parsed.date = to_Biden_parsed.date.apply(as_datetime)
from_Biden_parsed.date = from_Biden_parsed.date.apply(as_datetime)

to_Trump_parsed.date = to_Trump_parsed.date.apply(as_datetime)
from_Trump_parsed.date = from_Trump_parsed.date.apply(as_datetime)

to_Biden_parsed.date = to_Biden_parsed.date.astype('datetime64[ns]')
from_Biden_parsed.date = from_Biden_parsed.date.astype('datetime64[ns]')

to_Trump_parsed.date = to_Trump_parsed.date.astype('datetime64[ns]')
from_Trump_parsed.date = from_Trump_parsed.date.astype('datetime64[ns]')

In [148]:
to_Biden_parsed = to_Biden_parsed[to_Biden_parsed.date.dt.month>8]
from_Biden_parsed = from_Biden_parsed[from_Biden_parsed.date.dt.month>8]

to_Trump_parsed = to_Trump_parsed[to_Trump_parsed.date.dt.month>8]
from_Trump_parsed = from_Trump_parsed[from_Trump_parsed.date.dt.month>8]

__잘 적용되었나__

In [149]:
to_Biden_parsed.head(1)

,date,text
0,2020-09-02,Joe Haven't you been in government for like 47...


In [150]:
from_Biden_parsed.head(1)

,date,text
0,2020-09-01,I urge President Trump to join me in saying th...


In [151]:
to_Trump_parsed.head(1)

,date,text
0,2020-09-01,People are tired of watching the highly politi...


In [152]:
from_Trump_parsed.head(1)

,date,text
0,2020-09-02,True!트윗 인용하기Andrew Pollackwill always care for...


In [153]:
to_Biden_parsed.date.sort_values()

12      2020-09-01
5       2020-09-01
0       2020-09-02
15      2020-09-02
14      2020-09-02
           ...    
37563   2020-10-31
37564   2020-10-31
37565   2020-10-31
37566   2020-10-31
37568   2020-10-31
Name: date, Length: 978, dtype: datetime64[ns]

In [154]:
from_Biden_parsed.date.sort_values()

0      2020-09-01
1      2020-09-01
2      2020-09-01
3      2020-09-01
6      2020-09-01
          ...    
3315   2020-10-31
3316   2020-10-31
3317   2020-10-31
3318   2020-10-31
3320   2020-10-31
Name: date, Length: 526, dtype: datetime64[ns]

In [155]:
to_Trump_parsed.date.sort_values()

0       2020-09-01
103     2020-09-02
19      2020-09-02
18      2020-09-02
17      2020-09-02
           ...    
52038   2020-10-31
52042   2020-10-31
52040   2020-10-31
52041   2020-10-31
52043   2020-10-31
Name: date, Length: 968, dtype: datetime64[ns]

In [156]:
from_Trump_parsed.date.sort_values()

1       2020-09-01
3       2020-09-01
5       2020-09-01
7       2020-09-01
11      2020-09-01
           ...    
11556   2020-10-31
11557   2020-10-31
11559   2020-10-31
11561   2020-10-31
11563   2020-10-31
Name: date, Length: 705, dtype: datetime64[ns]

__저장 전 , 자잘한 작업__  
- 인덱스 리셋팅  
- 날짜별로 정렬한 상태로 저장하기 (그냥 보기 좋으라규)  

In [157]:
to_Biden_parsed = to_Biden_parsed.reset_index()
to_Biden_parsed = to_Biden_parsed.drop(['index'],axis=1)
to_Biden_parsed = to_Biden_parsed.sort_values(by='date')

from_Biden_parsed = from_Biden_parsed.reset_index()
from_Biden_parsed = from_Biden_parsed.drop(['index'],axis=1)
from_Biden_parsed = from_Biden_parsed.sort_values(by='date')

to_Trump_parsed = to_Trump_parsed.reset_index()
to_Trump_parsed = to_Trump_parsed.drop(['index'],axis=1)
to_Trump_parsed = to_Trump_parsed.sort_values(by='date')

from_Trump_parsed = from_Trump_parsed.reset_index()
from_Trump_parsed = from_Trump_parsed.drop(['index'],axis=1)
from_Trump_parsed = from_Trump_parsed.sort_values(by='date')

__내 폴더에 저어장__

In [158]:
to_Biden_parsed.to_csv(path+'to_Biden_with_hashtag.csv',index=False)
from_Biden_parsed.to_csv(path+'from_Biden_with_hashtag.csv',index=False)
to_Trump_parsed.to_csv(path+'to_Trump_with_hashtag.csv',index=False)
from_Trump_parsed.to_csv(path+'from_Trump_with_hashtag.csv',index=False)